<a href="https://colab.research.google.com/github/bascoul/Ml-Agents/blob/master/Part1_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="center">
  <td align="center"><a target="_blank" href="http://introtodeeplearning.com">
        <img src="http://introtodeeplearning.com/images/colab/mit.png" style="padding-bottom:5px;" />
      Visit MIT Deep Learning</a></td>
  <td align="center"><a target="_blank" href="https://colab.research.google.com/github/aamini/introtodeeplearning/blob/master/lab2/Part1_MNIST.ipynb">
        <img src="http://introtodeeplearning.com/images/colab/colab.png?v2.0"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/aamini/introtodeeplearning/blob/master/lab2/Part1_MNIST.ipynb">
        <img src="http://introtodeeplearning.com/images/colab/github.png"  height="70px" style="padding-bottom:5px;"  />View Source on GitHub</a></td>
</table>

# Copyright Information

In [0]:
# Copyright 2020 MIT 6.S191 Introduction to Deep Learning. All Rights Reserved.
# 
# Licensed under the MIT License. You may not use this file except in compliance
# with the License. Use and/or modification of this code outside of 6.S191 must
# reference:
#
# © MIT 6.S191: Introduction to Deep Learning
# http://introtodeeplearning.com
#

# Laboratoire 2: Computer Vision

# Partie 1 : Classification des chiffres MNIST

Dans la première partie de ce laboratoire, nous allons construire et former un réseau neuronal convolutif (CNN) pour la classification des chiffres manuscrits du célèbre ensemble de données [MNIST](http://yann.lecun.com/exdb/mnist/). L'ensemble de données du MNIST comprend 60 000 images d'entraînement et 10 000 images de test. Nos classes sont les chiffres 0-9.

Tout d'abord, téléchargeons le référentiel de cours, installons les dépendances et importons les paquets dont nous aurons besoin pour ce laboratoire.

In [0]:
# Importer Tensorflow 2.0
%tensorflow_version 2.x
import tensorflow as tf 

!pip install mitdeeplearning
import mitdeeplearning as mdl

import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm import tqdm

# Vérifier que nous utilisons un GPU, sinon changer de runtime
#   using Runtime > Change Runtime Type > GPU
assert len(tf.config.list_physical_devices('GPU')) > 0

## 1.1 L'ensemble de données MNIST 

Téléchargeons et chargeons l'ensemble de données et affichons quelques échantillons aléatoires de celui-ci :

In [0]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = (np.expand_dims(train_images, axis=-1)/255.).astype(np.float32)
train_labels = (train_labels).astype(np.int64)
test_images = (np.expand_dims(test_images, axis=-1)/255.).astype(np.float32)
test_labels = (test_labels).astype(np.int64)

Notre ensemble d'apprentissage est composé d'images en niveaux de gris 28x28 de chiffres manuscrits. 

Voyons à quoi ressemblent certaines de ces images et les étiquettes de formation correspondantes.

In [0]:
plt.figure(figsize=(10,10))
random_inds = np.random.choice(60000,36)
for i in range(36):
    plt.subplot(6,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    image_ind = random_inds[i]
    plt.imshow(np.squeeze(train_images[image_ind]), cmap=plt.cm.binary)
    plt.xlabel(train_labels[image_ind])

## 1.2 Réseau neuronal pour la classification des chiffres manuscrits

Nous allons d'abord construire un simple réseau de neurones composé de deux couches entièrement connectées et l'appliquer à la tâche de classification numérique. Notre réseau produira finalement une distribution de probabilité sur les classes à 10 chiffres (0-9). Cette première architecture que nous allons construire est représentée ci-dessous :

![alt_text](https://raw.githubusercontent.com/aamini/introtodeeplearning/master/lab2/img/mnist_2layers_arch.png "CNN Architecture for MNIST Classification")


### Architecture de réseau neuronal entièrement connecté
Pour définir l'architecture de ce premier réseau neuronal entièrement connecté, nous utiliserons à nouveau l'API de Keras et définirons le modèle à l'aide de la classe [`Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential). Notez que nous utilisons d'abord une couche [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), qui aplatit l'entrée afin qu'elle puisse être introduite dans le modèle. 

Dans le prochain bloc, vous allez définir les couches entièrement connectées de ce travail simple.

In [0]:
def build_fc_model():
  fc_model = tf.keras.Sequential([
      # Premièrement, définir la couche Flatten
      tf.keras.layers.Flatten(),

      # '''A FAIRE : Définissez la fonction d'activation de la première couche entièrement connectée (Dense).'''
      # Contrairement au laboratoire 1 nous n'utiliserons pas la fonction d'activation sigmoid mais la fonction
      # relu qui est à la fois plus simple en terme de calcul et meilleure pour l'optimisation
      tf.keras.layers.Dense(128, activation= '''A FAIRE'''),

      # '''A FAIRE : Définir la deuxième couche de densité pour produire les probabilités de classification'''
      # Keras permet de simplifier beaucoup la création du graphe mais il faut par contre bien connaitre les
      # différentes fonctions, comme nous souhaitons connaitre les 10 probabilités pour que notre chiffre
      # corresponde à 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, on utilise une activation de type softmax
      '''TODO: Couche dense pour produire des probabilités de classification'''
      
  ])
  return fc_model

model = build_fc_model()

Au fur et à mesure que nous avancerons dans cette prochaine partie, vous constaterez peut-être que vous voudrez apporter des modifications à l'architecture définie ci-dessus. **Notez que pour mettre à jour le modèle plus tard, vous devrez réexécuter la cellule ci-dessus pour réinitialiser le modèle. **

Prenons un peu de recul et pensons au réseau que nous venons de créer. La première couche de ce réseau, `tf.keras.layers.Flatten`, transforme le format des images d'une matrice de 2d (28 x 28 pixels), en une matrice de 1d de 28 * 28 = 784 pixels. On peut considérer ce calque comme un empilage de rangées de pixels dans l'image et leur alignement. Il n'y a pas de paramètres appris dans cette couche ; elle ne fait que reformater les données.

Une fois les pixels aplatis, le réseau est constitué d'une séquence de deux couches `tf.keras.layers.Dense`. Ce sont des couches neurales entièrement connectées. La première couche "dense" comporte 128 nœuds (ou neurones). La deuxième (et dernière) couche (que vous avez définie !) doit renvoyer un tableau de scores de probabilité dont la somme est égale à 1. Chaque noeud contient un score qui indique la probabilité que l'image actuelle appartienne à l'une des classes de chiffres manuscrits.

Cela définit notre modèle entièrement connecté ! 



### Compiler le modèle

Avant de former le modèle, nous devons définir quelques paramètres supplémentaires. Ceux-ci sont ajoutés lors de l'étape [`compile`] (https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#compile) du modèle :

* *Fonction de perte* - Cela définit la façon dont nous mesurons la précision du modèle pendant la formation. Comme nous l'avons vu en cours, nous voulons minimiser cette fonction pendant la formation, ce qui "orientera" le modèle dans la bonne direction.
* *Optimiseur* - Cela définit la façon dont le modèle est mis à jour en fonction des données qu'il voit et de sa fonction de perte.
* *Metrics* - Ici, nous pouvons définir les métriques utilisées pour contrôler les étapes de formation et de test. Dans cet exemple, nous allons examiner la *précision*, la fraction des images qui sont correctement classées.

Nous commencerons par utiliser un optimiseur de descente de gradient stochastique (SGD) initialisé avec un taux d'apprentissage de 0,1. Comme nous effectuons une tâche de classification catégorielle, nous allons utiliser la [perte d'entropie croisée] (https://www.tensorflow.org/api_docs/python/tf/keras/metrics/sparse_categorical_crossentropy).

Vous voudrez expérimenter à la fois le choix de l'optimiseur et le taux d'apprentissage et évaluer comment ceux-ci affectent la précision du modèle formé. 

In [0]:
'''TODO: Expérimentez avec différents optimiseurs et taux d'apprentissage.
    Comment ceux-ci affectent-ils la précision du modèle formé ? Quels sont les optimiseurs
    et/ou les taux d'apprentissage qui donnent les meilleures performances ?'''
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-1), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Former le modèle

Nous sommes maintenant prêts à former notre modèle, ce qui implique d'introduire les données d'apprentissage (`train_images` et `train_labels`) dans le modèle, puis de lui demander d'apprendre les associations entre les images et les étiquettes. Nous devrons également définir la taille du lot et le nombre d'époques, ou d'itérations sur le jeu de données MNIST, à utiliser pendant la formation. 

Dans le laboratoire 1, nous avons vu comment nous pouvons utiliser la "bande de gradient" pour optimiser les pertes et entraîner les modèles avec une descente de gradient stochastique. Après avoir défini les paramètres du modèle lors de l'étape de compilation, nous pouvons également effectuer l'entraînement en appelant la méthode [`fit`] (https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#fit) sur une instance de la classe `Model`. Nous utiliserons cette méthode pour entraîner notre modèle entièrement connecté


In [0]:
# Définir la taille du lot et le nombre d'époques à utiliser pendant la formation
BATCH_SIZE = 64
EPOCHS = 5

model.fit(train_images, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS)

Au fur et à mesure que le modèle s'entraîne, les mesures de perte et de précision sont affichées. Avec cinq époques et un taux d'apprentissage de 0,01, ce modèle entièrement connecté devrait atteindre une précision d'environ 0,97 (ou 97%) sur les données d'entraînement.

### Évaluer la précision de l'ensemble des données de test

Maintenant que nous avons formé le modèle, nous pouvons lui demander de faire des prédictions sur un ensemble de tests qu'il n'a jamais vu auparavant. Dans cet exemple, le tableau `test_images` comprend notre ensemble de données de test. Pour évaluer la précision, nous pouvons vérifier si les prédictions du modèle correspondent aux étiquettes du tableau `test_labels`. 

Utilisez la méthode [`evaluate`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#evaluate) pour évaluer le modèle sur l'ensemble de données de test !

In [0]:
'''TODO: Utiliser la méthode evaluate pour tester le modèle!'''
test_loss, test_acc = # TODO

print('Test accuracy:', test_acc)

Vous pouvez observer que la précision de l'ensemble de données de test est un peu plus faible que celle de l'ensemble de données de formation. Cet écart entre la précision de l'apprentissage et la précision du test est un exemple de *sur-apprentissage*, lorsqu'un modèle d'apprentissage sur machine a de moins bonnes performances sur les nouvelles données que sur ses données de formation. 

Quelle est la précision la plus élevée que vous pouvez obtenir avec ce premier modèle entièrement connecté ? Comme la tâche de classification des chiffres manuscrits est assez simple, vous vous demandez peut-être comment nous pouvons faire mieux...

![Deeper...](https://i.kym-cdn.com/photos/images/newsfeed/000/534/153/f87.jpg)

## 1.3 Réseau neural convolutif (CNN) pour la classification des chiffres manuscrits

Comme nous l'avons vu en conférence, les réseaux neuronaux convolutionnels (CNN) sont particulièrement bien adaptés à une variété de tâches en vision par ordinateur, et ont atteint des précisions quasi-parfaites sur l'ensemble des données du MNIST. Nous allons maintenant construire un CNN composé de deux couches convolutionnelles et de couches de mise en commun, suivies de deux couches entièrement connectées, et finalement produire une distribution de probabilité sur les classes à 10 chiffres (0-9). Le CNN que nous allons construire est décrit ci-dessous :

![alt_text](https://raw.githubusercontent.com/aamini/introtodeeplearning/master/lab2/img/convnet_fig.png "CNN Architecture for MNIST Classification")

### Définir le modèle CNN

Nous utiliserons les mêmes ensembles de données d'apprentissage et de test qu'auparavant, et procéderons de la même manière que pour notre réseau entièrement connecté pour définir et former notre nouveau modèle CNN. Pour ce faire, nous allons explorer deux couches que nous n'avons pas encore rencontrées : vous pouvez utiliser [`keras.layers.Conv2D` ](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) pour définir les couches convolutionnelles et [`keras.layers.MaxPool2D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) pour définir les couches de mise en commun. Utilisez les paramètres indiqués dans l'architecture du réseau ci-dessus pour définir ces couches et construire le modèle CNN.

In [0]:
def build_cnn_model():
    cnn_model = tf.keras.Sequential([

        # A FAIRE : Définir la première couche convolutive
        tf.keras.layers.Conv2D(filters='''A FAIRE ''', kernel_size=('''A FAIRE '''), activation='''A FAIRE '''),

        # A FAIRE : Définir la première couche de mise en commun max
        tf.keras.layers.MaxPool2D(pool_size=('''A FAIRE ''')),

        # A FAIRE : Définir la deuxième couche convolutive
        tf.keras.layers.Conv2D('''A FAIRE '''),

        # A FAIRE : Définir la deuxième couche de mise en commun max
        tf.keras.layers.MaxPool2D('''A FAIRE '''),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation=tf.nn.relu),

        # A FAIRE : Définissez la dernière couche de densité pour produire les 
        # probabilités de classification. Faites attention à l'activation
        # nécessaire à la sortie des probabilités
        # Déjà vu dans l'exemple précédent totalement connecté
        '''A FAIRE : Couche dense pour produire des probabilités de classification'''
    ])
    
    return cnn_model
  
cnn_model = build_cnn_model()
# Initialisez le modèle en faisant passer certaines données
cnn_model.predict(train_images[[0]])
# Imprimez le résumé des couches du modèle.
print(cnn_model.summary())

### Entraînez-vous et testez le modèle CNN

Maintenant, comme auparavant, nous pouvons définir la fonction de perte, l'optimiseur et les mesures par la méthode de la "compilation". Compilez le modèle CNN avec un optimiseur et un taux d'apprentissage de votre choix :

In [0]:
'''A FAIRE : Définissez l'opération de compilation avec l'optimiseur et le taux d'apprentissage de votre choix'''
cnn_model.compile(optimizer='''A FAIRE''', loss='''A FAIRE''', metrics=['accuracy']) # A FAIRE

Comme dans le cas du modèle entièrement connecté, nous pouvons former notre CNN en utilisant la méthode "fit" via l'API Keras.

In [0]:
'''A FAIRE : Utilisez model.fit pour former le modèle CNN, avec la même taille de lot et le même nombre d'époques que ceux utilisés précédemment.'''
cnn_model.fit('''A FAIRE''')

Super ! Maintenant que nous avons formé le modèle, évaluons-le sur l'ensemble des données de test en utilisant la méthode [`evaluate`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#evaluate) :

In [0]:
'''A FAIRE : Utilisez la méthode d'évaluation pour tester le modèle !'''
test_loss, test_acc = # A FAIRE

print('Test accuracy:', test_acc)

Quelle est la plus grande précision que vous pouvez obtenir en utilisant le modèle CNN, et comment la précision du modèle CNN se compare-t-elle à celle du simple réseau entièrement connecté ? Quels sont les optimiseurs et les taux d'apprentissage qui semblent être les meilleurs pour la formation au modèle CNN ? 

### Faites des prédictions avec le modèle CNN

Avec le modèle formé, nous pouvons l'utiliser pour faire des prédictions sur certaines images. L'appel de fonction [`predict`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#predict) génère les prédictions de sortie à partir d'un ensemble d'échantillons d'entrée.


In [0]:
predictions = cnn_model.predict(test_images)

Avec cet appel de fonction, le modèle a prédit l'étiquette pour chaque image du jeu de test. Examinons la prédiction pour la première image de l'ensemble de données de test :

In [0]:
predictions[0]

Comme vous pouvez le voir, une prédiction est un tableau de 10 chiffres. Rappelons que la sortie de notre modèle est une distribution de probabilité sur les classes à 10 chiffres. Ainsi, ces nombres décrivent la "confiance" du modèle dans le fait que l'image correspond à chacun des 10 chiffres différents. 

Examinons le chiffre qui a la plus grande confiance pour la première image dans l'ensemble de données de test :

In [0]:
'''A FAIRE : identifier le chiffre ayant la plus grande confiance dans la prévision de la première image
    de l'ensemble de données de test. '''
prediction = # A FAIRE

print(prediction)

Le modèle est donc convaincu que cette image est un " ? ??? ". Nous pouvons vérifier l'étiquette du test (n'oubliez pas qu'il s'agit de la véritable identité du chiffre) pour voir si cette prédiction est correcte :

In [0]:
print("Label of this digit is:", test_labels[0])
plt.imshow(test_images[0,:,:,0], cmap=plt.cm.binary)

C'est bon ! Visualisons les résultats de la classification sur l'ensemble des données du MNIST. Nous allons tracer les images de l'ensemble de données de test avec leur étiquette prédite, ainsi qu'un histogramme qui fournit les probabilités de prédiction pour chacun des chiffres :

In [0]:
#@title Changez le curseur pour voir les prédictions du modèle !{ run: "auto" }

image_index = 79 #@param {type:"slider", min:0, max:100, step:1}
plt.subplot(1,2,1)
mdl.lab2.plot_image_prediction(image_index, predictions, test_labels, test_images)
plt.subplot(1,2,2)
mdl.lab2.plot_value_prediction(image_index, predictions,  test_labels)

Nous pouvons également tracer plusieurs images avec leurs prédictions, où les étiquettes de prédiction correctes sont bleues et les étiquettes de prédiction incorrectes sont rouges. Le nombre donne le pourcentage de confiance (sur 100) pour l'étiquette prédite. Notez que le modèle peut être très confiant dans une prédiction incorrecte !

In [0]:
# Trace les premières images du test X, leur étiquette prédite et la véritable étiquette
# Prédictions correctes en bleu, prédictions incorrectes en rouge
num_rows = 5
num_cols = 4
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  mdl.lab2.plot_image_prediction(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  mdl.lab2.plot_value_prediction(i, predictions, test_labels)


## 1.4 Apprentissage du modèle 2.0

Plus tôt dans le laboratoire, nous avons utilisé l'appel de fonction [`fit`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#fit) pour entraîner le modèle. Cette fonction est de haut niveau et intuitive, ce qui est très utile pour les modèles plus simples. Comme vous pouvez le constater, cette fonction permet d'abstraire de nombreux détails dans l'appel de l'apprentissage, et nous avons moins de contrôle sur le modèle de formation, qui pourrait être utile dans d'autres contextes. 

Comme alternative, nous pouvons utiliser la classe [`tf.GradientTape`](https://www.tensorflow.org/api_docs/python/tf/GradientTape) pour enregistrer les opérations de différenciation pendant l'apprentissage, et ensuite appeler la fonction [`tf.GradientTape.gradient`](https://www.tensorflow.org/api_docs/python/tf/GradientTape#gradient) pour calculer réellement les gradients. Vous vous souvenez peut-être d'avoir vu cela dans le laboratoire 1, partie 1, mais regardons cela de plus près ici.

Nous utiliserons ce cadre pour entraîner notre `cnn_model` en utilisant la descente stochastique des gradients.

In [0]:
# Reconstruire le modèle CNN
cnn_model = build_cnn_model()

batch_size = 12
loss_history = mdl.util.LossHistory(smoothing_factor=0.95) # pour enregistrer l'évolution de la perte
plotter = mdl.util.PeriodicPlotter(sec=2, xlabel='Iterations', ylabel='Loss', scale='semilogy')
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2) # définir notre optimiseur

if hasattr(tqdm, '_instances'): tqdm._instances.clear() # nettoyer si elles existent

for idx in tqdm(range(0, train_images.shape[0], batch_size)):
  # Saisissez d'abord un lot de données d'apprentissage et convertissez les images d'entrée en tenseurs
  (images, labels) = (train_images[idx:idx+batch_size], train_labels[idx:idx+batch_size])
  images = tf.convert_to_tensor(images, dtype=tf.float32)

  # GradientTape pour enregistrer les opérations de dérivation
  with tf.GradientTape() as tape:
    #'''A FAIRE : introduire les images dans le modèle et obtenir les prédictions'''
    logits = # A FAIRE

    #'''A FAIRE : calculer la perte d'entropie croisée catégorique
    loss_value = tf.keras.backend.sparse_categorical_crossentropy() # TODO

  loss_history.append(loss_value.numpy().mean()) # ajouter la perte à l'enregistrement loss_history
  plotter.plot(loss_history.get())

  # Backpropagation
  '''A FAIRE : Utilisez la bande pour calculer le gradient par rapport à tous les paramètres du modèle CNN.
      Utilisez cnn_model.trainable_variables pour accéder à ces paramètres.''' 
  grads = # A FAIRE
  optimizer.apply_gradients(zip(grads, cnn_model.trainable_variables))


## 1.5 Conclusion
Dans cette partie du laboratoire, vous avez eu l'occasion de jouer avec différents classificateurs MNIST avec différentes architectures (couches entièrement connectées uniquement, CNN), et d'expérimenter comment différents hyperparamètres affectent la précision (taux d'apprentissage, etc.). La partie suivante du laboratoire explore une autre application des CNN, la détection faciale, et certains inconvénients des systèmes d'IA dans les applications du monde réel, comme les problèmes de biais. 